In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

In [ ]:
glb = glob.glob(os.getcwd()+"/dump/*.csv")
li = []
for filename in glb:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True).drop(columns='Unnamed: 0')

df.columns = ['artist','genre','song','lyrics']
df

In [ ]:
df.genre.unique()

In [ ]:
df.artist.unique()

In [ ]:
df.lyrics

In [ ]:
cv = CountVectorizer(stop_words=None, analyzer='word')
X = cv.fit_transform(df.lyrics)
X.toarray().sum(axis=0)

In [ ]:
cv.get_feature_names_out().shape

dff = pd.DataFrame(zip(X.toarray().sum(axis=0),cv.get_feature_names_out() )).sort_values(by=[0],ascending=False).loc[:5000]

from IPython.core.display import HTML
HTML(dff.loc[:20].to_html(escape=False))


In [ ]:

pipe = Pipeline (steps= [
    ('cv', CountVectorizer(stop_words='english', max_df=1, max_features=5000)),
    ('lda', LatentDirichletAllocation(learning_method='online',n_components=10, n_jobs=-1,random_state=123))
])

# pipe.fit(df.lyrics)
# words = pipe.named_steps.cv.get_feature_names_out()
# topics = pipe.named_steps.lda.components_ / pipe.named_steps.lda.components_.sum(axis=1)[:, np.newaxis]
# topicos = pd.DataFrame(topics, columns=words).T
# def call_top_topics(topics_df, topic_n, top_n):
#     return topics_df[topic_n].sort_values(ascending=False).head(top_n)


# call_top_topics(topicos, 0, 5000)  # POP Romantica ... Britney Spears


In [ ]:
# cv = CountVectorizer(stop_words='english', max_df=1, max_features=5000)
# transformed = cv.fit_transform(df.lyrics)

# n_components = 10
# learning_decay = .5



In [11]:
params = {
    'lda__n_components': [5, 10, 15],
    'lda__learning_decay': [.7, .5] 
    }

search = GridSearchCV(pipe, params, cv = 5, scoring = 'accuracy', n_jobs=-1, return_train_score=True, refit=True)
search.fit(df.lyrics)
search.best_estimator_.get_params()

In [ ]:
# mediante .components_ podemos extraer una matriz que entrega las distribución de palabras por cada tópico.
for topic_id, topic_name in enumerate(fit_best_lda.components_):
    # para cada tópico
    print("tópico: {}".format(topic_id + 1))
    # mediante argsort logramos ordenar los elementos por magnitud
    # para los elementos más relevantes ordenados por argsort, buscamos su correlativo
    # en la matriz dispersa y devolvemos el nombre.
    # finalmente concatenamos las palabras
    print(" ".join([counter.get_feature_names()[i] for i in topic_name.argsort()[:-15 - 1: -1]]))

In [ ]:
from sklearn.datasets import make_multilabel_classification
X, _ = make_multilabel_classification(random_state=0)
X

In [ ]:
df.lyrics